## Data Information

Data have 5 columns and more than 100 million rows. There are about 1 million users whoes user behaviors including click, purchase, adding item to shopping cart and item favoring during November 25 to December 03, 2017. Each line represents a specific user-item interaction, which consists of user ID, item ID, item's category ID, behavior type and timestamp, separated by commas.

In [1]:
# Modules imported
import pandas as pd
import numpy as np
import matplotlib as plt

In [2]:
data = pd.read_csv('/Users/paxton615/Downloads/UserBehavior.csv')

In [3]:
data.shape

(3835330, 5)

In [4]:
data.isnull().sum()

1             0
2268318       0
2520377       0
pv            0
1511544070    1
dtype: int64

In [5]:
data.nunique() # take 2-3 minutes to complete, be cautious to run this line

1              37376
2268318       930607
2520377         7106
pv                 5
1511544070    698888
dtype: int64

In [6]:
data.head()

,1,2268318,2520377,pv,1511544070
0,1,2333346,2520771,pv,1.511562e+09
1,1,2576651,149192,pv,1.511573e+09
2,1,3830808,4181361,pv,1.511593e+09
3,1,4365585,2520377,pv,1.511596e+09
4,1,4606018,2735466,pv,1.511616e+09


In [7]:
# col names = ['user_id', 'item_id','category_id','status',"timestamp"]
# pv: page view, buy: buy, cart: cart:, fav:favorite
data.columns = ['user_id', 'item_id','category_id','status',"timestamp"]

Trying some concert code here：

fun = data.head()
fun

str(pd.Timestamp(fun['timestamp'][0], unit='s'))[:10] 

convert str to timestamp
str(pd.Timestamp('2017-11-11').year)+'-'+str(pd.Timestamp('2017-11-11').month)

pd.Timestamp(fun['timestamp'][0], unit='s').hour

pd.Timestamp(fun['timestamp'][0], unit='s').month

It worked!

In [8]:
# select 3 million for analysis, calling them 'users'
users = data.iloc[5000000:8000000,:]

In [9]:
users.shape

(0, 5)

In [10]:
users.reset_index(drop=True, inplace=True)

In [11]:
users.head()

,user_id,item_id,category_id,status,timestamp


In [12]:
users.nunique()

user_id        0
item_id        0
category_id    0
status         0
timestamp      0
dtype: int64

In [13]:
# How much does ths sample, users, represent the population.
total_nuni = ['1 987994','2268318 4162024','2520377 9439','pv 4','1511544070 815859']

pop_unq = [int(i.split(' ')[1]) for i in total_nuni]
pop_unq

sample_unq = [i for i in users.nunique()]
sample_unq

perctage = []
for i,j in zip(sample_unq,pop_unq):
    perctage.append('{:2%}'.format(i/j))

# the subset, users, contains 3% of total users,
# 19% of total items, 73% of categories
perctage

['0.000000%', '0.000000%', '0.000000%', '0.000000%', '0.000000%']

In [14]:
users['hour']=[pd.Timestamp(i, unit='s',tz='Asia/Shanghai').hour for i in users.timestamp]

users['year']=[pd.Timestamp(i, unit='s',tz='Asia/Shanghai').year for i in users.timestamp]

users['day']=[pd.Timestamp(i, unit='s',tz='Asia/Shanghai').day for i in users.timestamp]

users['month']=[pd.Timestamp(i, unit='s',tz='Asia/Shanghai').month for i in users.timestamp]
# 0 is Monday，6 is Sunday
users['dayofweek']=[pd.Timestamp(i, unit='s',tz='Asia/Shanghai').dayofweek for i in users.timestamp]

In [15]:
users.year.unique() # data were generated during 2017-11-25 to 2017-12-3
# 2020,1919,2021 need to be found and cleaned later!

array([], dtype=float64)

In [16]:
users[(users['year']==1919)| (users['year']==2021)|(users['year']==2020)].count()

user_id        0
item_id        0
category_id    0
status         0
timestamp      0
hour           0
year           0
day            0
month          0
dayofweek      0
dtype: int64

In [17]:
users = users[users['year']==2017] # only choose 2017 data

In [18]:
np.sort(users.hour.unique()) # hours seem normal

array([], dtype=float64)

In [19]:
users.day.unique() 
# keep the days within 11.25-12.3

array([], dtype=float64)

In [20]:
users = users[users['day'].isin([25,26,27,28,29,30,1,2,3])] 

In [21]:
users.month.unique()
# data happened during 2017.11-2017.12, some months need to be cleaned

array([], dtype=float64)

In [22]:
users[(users['month']==10)| (users['month']==4)|(users['month']==9)|(users['month']==5)|(users['month']==8)].count()


user_id        0
item_id        0
category_id    0
status         0
timestamp      0
hour           0
year           0
day            0
month          0
dayofweek      0
dtype: int64

In [23]:
users = users[users['month'].isin([11,12])]

In [24]:
users.shape

(0, 10)

In [25]:
users.head() 

,user_id,item_id,category_id,status,timestamp,hour,year,day,month,dayofweek


In [26]:
users.to_csv(r'/Users/paxton615/Github_Personal/Alibaba_UserBehavior_Analysis/drafts/users_3m.csv')

In [27]:
# dropduplicstes: same user,same day, same hour, viewed the same item 
users = users.drop_duplicates(subset=['user_id','item_id','status','hour','day'])


In [28]:
users.shape

(0, 10)

In [29]:
users.head()

,user_id,item_id,category_id,status,timestamp,hour,year,day,month,dayofweek


In [30]:
# Form user['date'] by using year, month, day
date = []
for i,j,k in zip(users['year'], users['month'], users['day']):
    date.append(str(i)+"-"+str(j)+"-"+str(k))

users['date']=pd.to_datetime(date)

users.head()

,user_id,item_id,category_id,status,timestamp,hour,year,day,month,dayofweek,date


In [31]:
# tidy cols and reset_index
users = users[['user_id', 'item_id' , 'category_id', 'status', 'date','dayofweek' ,'hour']]
users.reset_index(drop=True, inplace=True)

In [32]:
users.dtypes
# Could convert hour to datetime as well, however, don't see the need yet
# pd.to_datetime(users['hour'], format='%H')

user_id                 int64
item_id                 int64
category_id             int64
status                 object
date           datetime64[ns]
dayofweek             float64
hour                  float64
dtype: object

In [33]:
users.head()

,user_id,item_id,category_id,status,date,dayofweek,hour


In [34]:
users.to_csv(r'/Users/paxton615/Github_Personal/Alibaba_UserBehavior_Analysis/drafts/users_2m.csv')

# Saved , 2.7m entries.

In [35]:
# Modules imported
import pandas as pd
import numpy as np
import matplotlib as plt

In [36]:
users = pd.read_csv('/Users/paxton615/Github_Personal/Alibaba_UserBehavior_Analysis/drafts/users_2m.csv')


In [37]:
users.head()

,Unnamed: 0,user_id,item_id,category_id,status,date,dayofweek,hour


In [38]:
users.drop('Unnamed: 0', axis=1, inplace=True)

In [39]:
users.head()

,user_id,item_id,category_id,status,date,dayofweek,hour


# 用户使用行为的漏斗模型

分析用户使用行为的漏斗模型，AARRR模型分析用户行为，此处数据主要涉及用户刺激和购买转化环节。
通过用户从浏览到最终购买过程的流失情况，包括浏览，收藏，加入购物车和购买。在本例中，各项指标如下:

In [40]:
# 访问用户总数 （UV）:29233
users.nunique()

user_id        0
item_id        0
category_id    0
status         0
date           0
dayofweek      0
hour           0
dtype: int64

In [41]:
# 页面总访问量 （PV）：2406344
users.groupby('status')['user_id'].count()

Series([], Name: user_id, dtype: int64)

平均每人每周访问量

In [44]:
# # Average PV per week per person
# avg_pv = users.groupby('status')['user_id'].count() / users.user_id.nunique()
# '{:.4}'.format(avg_pv)

跳失率：只点击一次浏览的用户数量/总用户访问量

user_stat_cnts0 = pd.DataFrame(users.groupby(['user_id','status'])['status'].count())

user_stat_cnts0.head()

user_stat_cnts0.loc[42,'pv']

user_stat_cnts1=user_stat_cnts0.reset_index(level=0)

user_stat_cnts1.head()

In [45]:
# 每个用户的pv数量是多少？逻辑：相当于通过按user_id来groupby，统计大类（status）中某个小类（pv）的数量
# 很重要的groupby应用实例，filter之后再做groupby
# 很重要，很重要
user_pv = pd.DataFrame(users[users['status']=='pv'].groupby(users['user_id'])['status'].count())
user_pv.columns=['pv']
user_pv.head()

,pv
user_id,


In [46]:
# 只有一次浏览记录的用户有多少？
# 可见，非常少，可以忽略不计。
len(user_pv[user_pv['pv'] == 1])

0

In [47]:
# 所有user的总浏览量
users[users['status']=='pv']['user_id'].count()

0

In [48]:
# 跳失率
Bounce_rate = len(user_pv[user_pv['pv'] == 1]) / users[users['status']=='pv']['user_id'].count()

'{:.6%}'.format(Bounce_rate)


/Users/paxton615/pycharm/projects/venv/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in long_scalars
  


'nan%'

漏斗模型：
喜欢和放入购物车都属于决策环节，二者合一。
百分之多少的人从pv进入了cart和fav环节
又有百分之多少的人从cart和fav环节进入了最后的buy

In [49]:
users_status = users.groupby('status').user_id.count()
users_status

Series([], Name: user_id, dtype: int64)

In [50]:
# 约10.46%的用户从浏览环节进入决策环节
pv_to_cartnfav = (users_status[1]+users_status[2])/users_status[3]

'{:.2%}'.format(pv_to_cartnfav)

IndexError: index out of bounds

In [51]:
# 约23.06%的用户从决策环节进入了购买环节，环比之前大幅提升
cartnfav_to_buy = users_status[0]/(users_status[1]+users_status[2])
'{:.3%}'.format(cartnfav_to_buy)

IndexError: index out of bounds

# 不同时间尺度下，用户行为模式分析

1.分析一周内的用户使用情况

In [52]:
users.date.unique()

array([], dtype=object)

In [53]:
# ub is user behavior
ub_daily_old=pd.DataFrame(users.groupby(['date','status'], as_index=False).user_id.count())
ub_daily_old.head()


,date,status,user_id


In [54]:
# .pivot() 以...为中心旋转 很重要很重要很重要
ub_daily = ub_daily_old.pivot(index='date',columns='status')
ub_daily

date


In [55]:
# 好像可以对multi_colums的col name直接做修改
ub_daily.columns = ['buy', 'cart', 'fav', 'pv']



ValueError: Length mismatch: Expected axis has 0 elements, new values have 4 elements

In [56]:
# 改名后调整cols的顺序
ub_daily = ub_daily[['pv','fav','cart','buy']]
ub_daily

ValueError: zero-size array to reduction operation maximum which has no identity

小结论：
11-29为周三，11-25为周六，12-03为周日
周一到周五数据趋于稳定，周末数据有明显增加，运营活动可向周末时间倾斜

2. 分析用户一天内的使用情况

In [57]:
users.head()

,user_id,item_id,category_id,status,date,dayofweek,hour


In [58]:
nov_28 = users[users['date']=='2017-11-29'][['user_id','status','hour']]

In [59]:
nov_28.head()

,user_id,status,hour


In [60]:
temp = nov_28.groupby(['hour','status'],as_index=False).count()

In [61]:
temp.head()

,hour,status,user_id


In [62]:
nov_28 = temp.pivot(index='hour',columns='status')

In [63]:
nov_28.head()

hour


In [64]:
nov_28.columns=['buy', 'cart', 'fav', 'pv']

ValueError: Length mismatch: Expected axis has 0 elements, new values have 4 elements

In [65]:
nov_28 = nov_28[['pv','fav','cart','buy']]
nov_28.head()

ValueError: zero-size array to reduction operation maximum which has no identity

# 此处发现一个大问题，11-28号，简书的案例中显示21，22点是用户最活跃的时间，而我的timestamp读取出的时间是13，14点！！！需返回后重新检查... ... (已返回，且问题已通过加入上海时区而解决)

In [66]:
# 设计一个function，将制定数据自动转换成某日的四种状态浏览记录
def daily_status(df,str_colname1,str_colname2,str_colname3):
    temp = df.groupby([str_colname3,str_colname2],as_index=False).count()
    daily_num = temp.pivot(index=str_colname3,columns=str_colname2)
    daily_num.columns=['buy', 'cart', 'fav', 'pv']
    daily_num = daily_num[['pv','fav','cart','buy']]
    return (daily_num)

In [67]:
nov_29=users[users['date']=='2017-11-29'][['user_id','status','hour']]

In [68]:
daily_status(nov_29,'user_id','status','hour').sort_values('pv',ascending=False).head(7)

ValueError: Length mismatch: Expected axis has 0 elements, new values have 4 elements

In [69]:
# "2017-12-02" correct ; "2017-12-2" incorrect
dec_2=users[users['date']=='2017-12-02'][['user_id','status','hour']]

In [70]:
daily_status(dec_2,'user_id','status','hour').sort_values('pv',ascending=False).head(7)

ValueError: Length mismatch: Expected axis has 0 elements, new values have 4 elements

小结论：从每天的数据情况来看，20-23是绝对使用高峰期，应该在此时间段内增加互动营销，以希望更大收益

# 不同商品种类的用户行为

1. 统计浏览次数，购买次数，收藏次数，和加入购物车次数最多的商品

In [71]:
users.head()

,user_id,item_id,category_id,status,date,dayofweek,hour


In [72]:
# top item been seen
users[users['status']=='pv'][['item_id','user_id']].groupby('item_id').count().sort_values('user_id',ascending=False).head()

,user_id
item_id,


In [73]:
# top items been bought
users[users['status']=='buy'][['item_id','user_id']].groupby('item_id').count().sort_values('user_id',ascending=False).head(20)

,user_id
item_id,


In [74]:
# top items in fav
users[users['status']=='fav'][['item_id','user_id']].groupby('item_id').count().sort_values('user_id',ascending=False).head(20)

,user_id
item_id,


In [75]:
# top items in cart
users[users['status']=='cart'][['item_id','user_id']].groupby('item_id').count().sort_values('user_id',ascending=False).head(20)

,user_id
item_id,


In [76]:
# 以上四幅图需要一个能有效结合在一起的方式，没有比较，几幅图的意义不大。
# 最好是图形的。

2. 统计所有商品的购买次数

In [77]:
users.head()

,user_id,item_id,category_id,status,date,dayofweek,hour


In [78]:
users[users['status']=='buy'][['item_id','user_id']].head()

,item_id,user_id


In [79]:
item_buytimes = users[users['status']=='buy'][['item_id','user_id']].groupby('item_id', as_index=False).count().sort_values('user_id',ascending=False)

In [80]:
item_buytimes.head()

,item_id,user_id


In [81]:
# There are 38449 kinds of iterms that only been bought once
# only one item was bought 34 times
item_buytimes.groupby('user_id').count()

,item_id
user_id,


In [82]:
item_buytimes_stas = pd.DataFrame(item_buytimes.groupby('user_id').count())

In [83]:
'{:.2%}'.format(item_buytimes_stas.item_id[1]/sum(item_buytimes_stas.item_id))

IndexError: index out of bounds

84.34%这个数据说明，大部分的销售来源于长尾商品的累积效应。而非爆款商品的带动。

# 基于RFM理论找出有价值的用户

In [84]:
users.head()

,user_id,item_id,category_id,status,date,dayofweek,hour


In [85]:
len(users[users['status']=='buy'].groupby('user_id',as_index=False)['date'].agg(['max']))

0

In [86]:
# the most recent shopping record of each user
recent_buy = users[users['status']=='buy'].groupby('user_id',as_index=False)['date'].agg(['max'])

In [87]:
recent_buy.head()

,max
user_id,


In [88]:
recent_buy.columns=['latest_shopping']
recent_buy.reset_index(inplace=True)

In [89]:
recent_buy.head()

,user_id,latest_shopping


In [90]:
np.sort(recent_buy.latest_shopping.unique())

array([], dtype=float64)

In [91]:
recent_buy['latest_shopping'][1]>'2017-11-02'

IndexError: index out of bounds

In [92]:
recent_buy['R'] = [0 if '2017-11-25'<= i <='2017-11-27' 
                   else 1 if '2017-11-28' <= i <= '2017-11-30' 
                   else 2 for i in recent_buy.latest_shopping]




In [93]:
recent_buy.head()

,user_id,latest_shopping,R


In [94]:
shopping_freq = users[users['status']=='buy'].groupby('user_id', as_index=False).item_id.count()

In [95]:
shopping_freq.head()

,user_id,item_id


In [96]:
shopping_freq[shopping_freq['user_id']==339189]

,user_id,item_id


In [97]:
shopping_freq.item_id.unique()

array([], dtype=int64)

In [98]:
shopping_freq['F'] = [ 5 if i >= 50 
                     else 4 if 49 >= i >= 40
                     else 3 if 39 >= i >= 30
                     else 2 if 29 >= i >= 20
                     else 1 for i in shopping_freq.item_id]

In [99]:
shopping_freq.head()

,user_id,item_id,F


In [100]:
shopping_freq[shopping_freq['user_id']==337305]

,user_id,item_id,F


In [101]:
rfm = pd.merge(recent_buy, shopping_freq, on='user_id')[['user_id','latest_shopping','R','F']]

In [102]:
rfm['R+F'] = rfm['R']+rfm['F']

In [103]:
rfm.head()

,user_id,latest_shopping,R,F,R+F


In [104]:
rfm[rfm['user_id']==337305]

,user_id,latest_shopping,R,F,R+F


In [105]:
rfm['R+F'].value_counts(normalize=True)

Series([], Name: R+F, dtype: float64)

In [106]:
1.5/11.4

0.13157894736842105

In [107]:
7.2/22

0.32727272727272727